In [1]:
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, ConcatDataset
import torch
from torchvision import datasets
from torchvision.transforms import transforms

In [2]:
#import os

In [3]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [4]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = nn.Sequential(
            BasicBlock(64, 64),
            BasicBlock(64, 64)
        )
        self.layer2 = nn.Sequential(
            BasicBlock(64, 128, stride=2),
            BasicBlock(128, 128)
        )
        self.layer3 = nn.Sequential(
            BasicBlock(128, 256, stride=2),
            BasicBlock(256, 256)
        )
        self.layer4 = nn.Sequential(
            BasicBlock(256, 512, stride=2),
            BasicBlock(512, 512)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [5]:
def train_model(epochs,model,train_load,val_load,test_load,loss_func,gradient_descent):
    train_loss=[]
    training_accuracy=[]
    validation_loss=[]
    validation_accuracy=[]
    testing_accuracy=[]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=model.to(device)
    for i in range(epochs):
        training_loss=0
        correct_predicted=0
        model.train()
        for image,label in train_load:
            image,label=image.to(device),label.to(device)
            output=model(image)
            loss=loss_func(output,label)
            gradient_descent.zero_grad()
            loss.backward()
            gradient_descent.step()
            training_loss+=loss.item()*len(image)
            dummy,predicted=torch.max(output.data,1)
            correct_predicted+=(predicted==label).sum().item()
        train_loss.append(training_loss/len(train_load.dataset))
        training_accu=correct_predicted/len(train_load.dataset)
        training_accuracy.append(training_accu)


        model.eval()
        validate_loss=0
        val_correct_predicted=0
        for image,label in val_load:
            image,label=image.to(device),label.to(device)
            output=model(image)
            loss=loss_func(output,label)
            gradient_descent.zero_grad()
            loss.backward()
            gradient_descent.step()
            validate_loss+=loss.item()*len(image)
            dummy,predicted=torch.max(output.data,1)
            val_correct_predicted+=(predicted==label).sum().item()
        validation_loss.append(validate_loss/len(val_load.dataset))
        validation_accu=val_correct_predicted/len(val_load.dataset)
        validation_accuracy.append(validation_accu)


        model.eval()
        test_correct_predicted=0
        with torch.no_grad():
            for image,label in test_load:
                image,label=image.to(device),label.to(device)
                output=model(image)
                dummy,predicted=torch.max(output.data,1)
                test_correct_predicted+=(predicted==label).sum().item()
        testing_accu_base_model=test_correct_predicted/len(test_load.dataset) 
        testing_accuracy.append(testing_accu_base_model)
        print(f'Epoch: {i+1} --------- Train Loss: {training_loss/len(train_load.dataset):.4f} --------- Train Accuracy: {training_accu:.4f} --------- Validation Loss: {validate_loss/len(val_load.dataset):.4f} --------- Validation Accuracy: {validation_accu:.4f}------------ Testing Accuracy: {testing_accu_base_model:.4f}')

    return model,train_loss,training_accuracy,validation_loss,validation_accuracy,testing_accuracy

In [6]:
def accuracy(test_loader, model):
    test_correct_predicted=0
    model.eval()
    with torch.no_grad():
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model=model.to(device)
        for image,label in test_loader:
            image,label=image.to(device),label.to(device)
            output=model(image)
            dummy,predicted=torch.max(output.data,1)
            test_correct_predicted+=(predicted==label).sum().item()
    testing_accu=test_correct_predicted/len(test_loader.dataset)         
    testing_accu

In [7]:
import torchvision.transforms as tt
train_tfms = tt.Compose([
                         tt.Grayscale(num_output_channels=3), # Pictures black and white
                         tt.Resize([128, 128]),
                         # Settings for expanding the dataset
                         tt.RandomHorizontalFlip(),           # Random 90 degree rotations
                         tt.RandomRotation(30),               # Random 30 degree rotations
                         tt.ToTensor(),                      # Cast to tensor
                         ])                      

test_tfms = tt.Compose([
                        tt.Grayscale(num_output_channels=3),
                        tt.Resize([128, 128]),
                        tt.ToTensor(),
                        ])
     

In [8]:
import torch
from torch.utils.data import random_split

dataset = ImageFolder("leapGestRecog",transform=transforms.ToTensor()) # Replace with your dataset

train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_set, val_set, test_set = random_split(dataset,[train_size, val_size, test_size])

In [9]:
train_loader=torch.utils.data.DataLoader(train_set,batch_size=256)
test_loader=torch.utils.data.DataLoader(test_set,batch_size=256)
val_loader=torch.utils.data.DataLoader(val_set,batch_size=256)

In [10]:
model1=ResNet18(10)
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model1.parameters(),lr=1e-4)
returns = train_model(epochs=10,model=model1,train_load=train_loader,val_load=val_loader,test_load=test_loader,loss_func=loss_function,gradient_descent=optimizer)

: 

: 